In [31]:
from cmdstanpy import cmdstan_path, set_cmdstan_path, CmdStanModel
import os
import numpy as np
set_cmdstan_path('/stan/math_HOW-TO-USE/cmdstan-ddm-7pm')
cmdstan_path()
import pandas as pd

Compile the model

In [62]:
stan_file = os.path.join('../models/', 'wiener_simple_boundary.stan')
ndt_full_model = CmdStanModel(stan_file=stan_file, cpp_options={'STAN_THREADS': True}, force_compile=True)

13:06:59 - cmdstanpy - INFO - compiling stan file /stan/math_HOW-TO-USE/cmdstan-ddm-7pm/nets/notebooks/jointError/models/wiener_simple_boundary.stan to exe file /stan/math_HOW-TO-USE/cmdstan-ddm-7pm/nets/notebooks/jointError/models/wiener_simple_boundary
13:07:17 - cmdstanpy - INFO - compiled model executable: /stan/math_HOW-TO-USE/cmdstan-ddm-7pm/nets/notebooks/jointError/models/wiener_simple_boundary


In [63]:
ndt_full_model.exe_info()

{'stan_version_major': '2',
 'stan_version_minor': '31',
 'stan_version_patch': '0',
 'STAN_THREADS': 'true',
 'STAN_MPI': 'false',
 'STAN_OPENCL': 'false',
 'STAN_NO_RANGE_CHECKS': 'false',
 'STAN_CPP_OPTIMS': 'false'}

In [65]:
os.environ['STAN_NUM_THREADS'] = "12"

Define data

In [25]:
data_file = os.path.join('../data/', 'stan_data_cleaned.data.json')

Read the original data

## Read and prepare data

In [39]:
df = pd.read_csv('../data/twentythree_participants_post_eeg_test_set_with_sequence_global_rt_thresholds.csv').drop(columns='Unnamed: 0')

# check dataframe
display(df.isnull().any())
display(df)

trial_number               False
ID                         False
participant_index          False
condition                   True
condition_index             True
rt                          True
acc                         True
ne_Fz                       True
ne_FCz                      True
ne_mean_Fz                  True
ne_mean_FCz                 True
y                           True
pre_ne_Fz                   True
pre_ne_FCz                  True
pre_ne_mean_Fz              True
pre_ne_mean_FCz             True
pre_acc                     True
rt_greater_than_1          False
log_rt_exceed_threshold    False
is_in_sequence             False
dtype: bool

,trial_number,ID,participant_index,condition,condition_index,rt,acc,ne_Fz,ne_FCz,ne_mean_Fz,ne_mean_FCz,y,pre_ne_Fz,pre_ne_FCz,pre_ne_mean_Fz,pre_ne_mean_FCz,pre_acc,rt_greater_than_1,log_rt_exceed_threshold,is_in_sequence
0,1,169,1,1.0,1.0,0.268716,1.0,-10.473746,-13.920862,-6.156650,-5.222685,0.268716,NaN,NaN,NaN,NaN,NaN,False,False,False
1,2,169,1,1.0,1.0,0.309487,1.0,-0.501844,-2.783326,3.766289,9.476191,0.309487,-10.473746,-13.920862,-6.156650,-5.222685,1.0,False,False,False
2,3,169,1,-1.0,2.0,0.397239,1.0,-3.449400,-3.684688,-1.556821,2.632504,0.397239,-0.501844,-2.783326,3.766289,9.476191,1.0,False,False,False
3,4,169,1,1.0,1.0,0.309485,1.0,0.790637,0.747574,4.928352,6.436701,0.309485,-3.449400,-3.684688,-1.556821,2.632504,1.0,False,False,True
4,5,169,1,-1.0,2.0,0.386673,1.0,-2.980749,-5.791032,-0.957992,-1.689976,0.386673,0.790637,0.747574,4.928352,6.436701,1.0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6895,296,208,23,-1.0,2.0,0.239623,-1.0,8.320652,4.675662,12.951381,12.453468,-0.239623,7.714413,8.938596,15.386128,15.872547,1.0,False,False,True
6896,297,208,23,-1.0,2.0,0.382499,1.0,2.655430,4.103194,5.651438,9.170830,0.382499,8.320652,4.675662,12.951381,12.453468,-1.0,False,False,True
6897,298,208,23,1.0,1.0,0.236665,1.0,-5.139061,-4.319686,-0.687883,-0.221055,0.236665,2.655430,4.103194,5.651438,9.170830,1.0,False,False,False
6898,299,208,23,1.0,1.0,0.261338,1.0,1.264665,0.551756,3.307084,4.809125,0.261338,-5.139061,-4.319686,-0.687883,-0.221055,1.0,False,False,False


Remove trials with NaNs

In [40]:
df_no_nans = df.dropna()

# check dataframe
display(df_no_nans.isnull().any())
display(df_no_nans)

trial_number               False
ID                         False
participant_index          False
condition                  False
condition_index            False
rt                         False
acc                        False
ne_Fz                      False
ne_FCz                     False
ne_mean_Fz                 False
ne_mean_FCz                False
y                          False
pre_ne_Fz                  False
pre_ne_FCz                 False
pre_ne_mean_Fz             False
pre_ne_mean_FCz            False
pre_acc                    False
rt_greater_than_1          False
log_rt_exceed_threshold    False
is_in_sequence             False
dtype: bool

,trial_number,ID,participant_index,condition,condition_index,rt,acc,ne_Fz,ne_FCz,ne_mean_Fz,ne_mean_FCz,y,pre_ne_Fz,pre_ne_FCz,pre_ne_mean_Fz,pre_ne_mean_FCz,pre_acc,rt_greater_than_1,log_rt_exceed_threshold,is_in_sequence
1,2,169,1,1.0,1.0,0.309487,1.0,-0.501844,-2.783326,3.766289,9.476191,0.309487,-10.473746,-13.920862,-6.156650,-5.222685,1.0,False,False,False
2,3,169,1,-1.0,2.0,0.397239,1.0,-3.449400,-3.684688,-1.556821,2.632504,0.397239,-0.501844,-2.783326,3.766289,9.476191,1.0,False,False,False
3,4,169,1,1.0,1.0,0.309485,1.0,0.790637,0.747574,4.928352,6.436701,0.309485,-3.449400,-3.684688,-1.556821,2.632504,1.0,False,False,True
4,5,169,1,-1.0,2.0,0.386673,1.0,-2.980749,-5.791032,-0.957992,-1.689976,0.386673,0.790637,0.747574,4.928352,6.436701,1.0,False,False,True
5,6,169,1,1.0,1.0,0.316943,1.0,2.572565,-1.050971,4.401086,7.993626,0.316943,-2.980749,-5.791032,-0.957992,-1.689976,1.0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6895,296,208,23,-1.0,2.0,0.239623,-1.0,8.320652,4.675662,12.951381,12.453468,-0.239623,7.714413,8.938596,15.386128,15.872547,1.0,False,False,True
6896,297,208,23,-1.0,2.0,0.382499,1.0,2.655430,4.103194,5.651438,9.170830,0.382499,8.320652,4.675662,12.951381,12.453468,-1.0,False,False,True
6897,298,208,23,1.0,1.0,0.236665,1.0,-5.139061,-4.319686,-0.687883,-0.221055,0.236665,2.655430,4.103194,5.651438,9.170830,1.0,False,False,False
6898,299,208,23,1.0,1.0,0.261338,1.0,1.264665,0.551756,3.307084,4.809125,0.261338,-5.139061,-4.319686,-0.687883,-0.221055,1.0,False,False,False


Remove trials with RT < 100ms for model to converge (problem with non-decision time)

In [41]:
# threshold = 0.1
# df_rts_truncated = df_no_nans[df_no_nans['rt'] > threshold]
# 
# df_rts_truncated

In [42]:
df_rts_truncated = df_no_nans

Filter CCXP trials sequences

In [43]:
df_rts_truncated = df_rts_truncated[df_rts_truncated['is_in_sequence'] == True]
df_rts_truncated

,trial_number,ID,participant_index,condition,condition_index,rt,acc,ne_Fz,ne_FCz,ne_mean_Fz,ne_mean_FCz,y,pre_ne_Fz,pre_ne_FCz,pre_ne_mean_Fz,pre_ne_mean_FCz,pre_acc,rt_greater_than_1,log_rt_exceed_threshold,is_in_sequence
3,4,169,1,1.0,1.0,0.309485,1.0,0.790637,0.747574,4.928352,6.436701,0.309485,-3.449400,-3.684688,-1.556821,2.632504,1.0,False,False,True
4,5,169,1,-1.0,2.0,0.386673,1.0,-2.980749,-5.791032,-0.957992,-1.689976,0.386673,0.790637,0.747574,4.928352,6.436701,1.0,False,False,True
5,6,169,1,1.0,1.0,0.316943,1.0,2.572565,-1.050971,4.401086,7.993626,0.316943,-2.980749,-5.791032,-0.957992,-1.689976,1.0,False,False,True
6,7,169,1,1.0,1.0,0.278136,1.0,2.280077,0.960363,11.493082,9.934797,0.278136,2.572565,-1.050971,4.401086,7.993626,1.0,False,False,True
7,8,169,1,1.0,1.0,0.316809,1.0,-6.511949,-8.959696,4.500986,7.554044,0.316809,2.280077,0.960363,11.493082,9.934797,1.0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6893,294,208,23,1.0,1.0,0.194927,1.0,-4.353022,-2.084978,-1.871309,1.272531,0.194927,-1.587728,-2.883128,6.603760,6.394492,1.0,False,False,True
6894,295,208,23,1.0,1.0,0.211838,1.0,7.714413,8.938596,15.386128,15.872547,0.211838,-4.353022,-2.084978,-1.871309,1.272531,1.0,False,False,True
6895,296,208,23,-1.0,2.0,0.239623,-1.0,8.320652,4.675662,12.951381,12.453468,-0.239623,7.714413,8.938596,15.386128,15.872547,1.0,False,False,True
6896,297,208,23,-1.0,2.0,0.382499,1.0,2.655430,4.103194,5.651438,9.170830,0.382499,8.320652,4.675662,12.951381,12.453468,-1.0,False,False,True


Prepare 1D data for Stan with information on per participant number of trials

In [44]:
y = df_rts_truncated['y'].to_numpy()
condition = df_rts_truncated['condition'].to_numpy()
pre_acc = df_rts_truncated['pre_acc'].to_numpy()
# pre_ne = df_rts_truncated['pre_ne_FCz_centered'].to_numpy()
participant_index = df_rts_truncated['participant_index'].to_numpy()

n_participants = len(np.unique(participant_index))
n_conditions = len(np.unique(condition))

participants_trials_slices = []
pre_ne = []
for index in np.unique(participant_index):
    indices = np.where(participant_index == index)[0]
    start_index = indices[0] + 1
    end_index = indices[-1] + 1
    participants_trials_slices.append([start_index, end_index])
    
    participants_ne = df_rts_truncated.iloc[indices]['pre_ne_FCz'].to_numpy().flatten()
    participants_ne_stand = (participants_ne - np.mean(participants_ne)) / np.std(participants_ne)
    
    pre_ne.extend(participants_ne_stand)
    
participants_trials_slices = np.array(participants_trials_slices)
pre_ne = np.array(pre_ne)
df_rts_truncated['pre_ne_FCz_centered'] = pre_ne

/tmp/ipykernel_3306/3348809180.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rts_truncated['pre_ne_FCz_centered'] = pre_ne


Fit the model

In [26]:
num_chains = 4
warmup = 1000
num_samples = 1000
thin=1

n_participants = 23 # mock

In [45]:
initials = []

min_rt = np.zeros(n_participants)
participant_index = df_rts_truncated['participant_index'].to_numpy()

for idx, participant_idx in enumerate(np.unique(participant_index)):
    participant_data = df_rts_truncated[df_rts_truncated['participant_index'] == participant_idx]['y'].to_numpy()
    min_rt[idx] = np.min(abs(participant_data))

print(min_rt)

for c in range(0, num_chains):
    chain_init = {               
        'ter_sd': np.random.uniform(.01, .2),
        'alpha_sd': np.random.uniform(.01, 1.),
        'alpha_cond_sd': np.random.uniform(.01, 1.),
        'delta_sd': np.random.uniform(.1, 3.),
        'delta_cond_sd': np.random.uniform(.1, 3.),
        
        'alpha_ne_sd': np.random.uniform(.01, .2), 
    
        'ter': np.random.uniform(0.0, .3),
        'alpha': np.random.uniform(1, 2), 
        'alpha_cond': np.random.uniform(-.5, .5),
        'delta': np.random.uniform(-4., 4.),
        'delta_cond': np.random.uniform(-4., 4.),

        'alpha_ne': np.random.uniform(-.05, .05),
        'alpha_pre_acc': np.random.uniform(-0.1, .1), 
        'alpha_ne_pre_acc': np.random.uniform(-.05, .05), 
        'alpha_ne_cond': np.random.uniform(-.05, .05), 
        'alpha_pre_acc_cond': np.random.uniform(-0.1, .1), 
        'alpha_ne_pre_acc_cond': np.random.uniform(-.05, .05), 
        
        'participants_ter': np.random.uniform(0.0, .3, size=n_participants),
        'participants_alpha': np.random.uniform(1, 2., size=n_participants), 
        'participants_alpha_cond': np.random.uniform(-0.5, .5, size=n_participants),
        'participants_delta': np.random.uniform(-4., 4., size=n_participants),
        'participants_delta_cond': np.random.uniform(-4., 4., size=n_participants),
        
        'participants_alpha_ne': np.random.uniform(-.05, .05, size=n_participants), 
    }

    for p in range(0, n_participants):
        chain_init['participants_ter'][p] = np.random.uniform(0., min_rt[p]/2)

    initials.append(chain_init)


[0.11573601 0.20435691 0.18330002 0.13231993 0.12545013 0.118536
 0.13919401 0.11866999 0.17839289 0.17711496 0.16358399 0.12372184
 0.17407513 0.11877799 0.17403293 0.16509008 0.12201595 0.11527514
 0.15353394 0.20710111 0.130481   0.21383595 0.16710591]


In [71]:
fit = ndt_full_model.sample(
    data=data_file,
    chains=num_chains, 
    seed=42, 
    inits=initials, 
    iter_warmup=warmup, 
    iter_sampling=num_samples,
    parallel_chains=4,
    threads_per_chain= 3,
    show_progress=True,
)

13:14:17 - cmdstanpy - INFO - CmdStan start processing
13:25:20 - cmdstanpy - ERROR - CmdStan error: terminated by signal 2 Unknown error -2


KeyboardInterrupt: 

In [59]:
fit.draws_pd()

,chain__,iter__,draw__,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,...,log_lik[5203],log_lik[5204],log_lik[5205],log_lik[5206],log_lik[5207],log_lik[5208],log_lik[5209],log_lik[5210],log_lik[5211],log_lik[5212]
0,1.0,1.0,1.0,-2289.49,6.669160e-01,0.007676,3.0,15.0,1.0,2743.120,...,1.54263,1.80611,1.77409,1.72395,1.74670,1.674170,1.79418,-1.489200,0.040456,-1.642160
1,1.0,2.0,2.0,-191.92,9.225110e-01,0.007676,4.0,15.0,0.0,2269.600,...,1.70001,1.59600,1.75460,1.75577,1.78739,1.119840,1.49639,-0.284065,0.267266,-0.261760
2,1.0,3.0,3.0,-191.92,3.054010e-20,0.007676,4.0,15.0,0.0,269.993,...,1.70001,1.59600,1.75460,1.75577,1.78739,1.119840,1.49639,-0.284065,0.267266,-0.261760
3,1.0,4.0,4.0,-191.92,4.897270e-19,0.007676,3.0,7.0,0.0,254.660,...,1.70001,1.59600,1.75460,1.75577,1.78739,1.119840,1.49639,-0.284065,0.267266,-0.261760
4,1.0,5.0,5.0,-191.92,6.389070e-20,0.007676,4.0,15.0,0.0,284.940,...,1.70001,1.59600,1.75460,1.75577,1.78739,1.119840,1.49639,-0.284065,0.267266,-0.261760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2.0,46.0,96.0,6401.88,6.754700e-01,0.006784,7.0,127.0,0.0,-6323.040,...,1.79269,2.34059,2.40830,2.31702,2.33055,1.376300,2.21154,0.534223,0.131221,0.579983
96,2.0,47.0,97.0,6391.55,8.320360e-01,0.006784,7.0,127.0,0.0,-6318.460,...,1.80268,2.29949,2.35664,2.26699,2.29175,1.515240,2.19900,0.528336,0.083868,0.655881
97,2.0,48.0,98.0,6397.17,9.683340e-01,0.006784,7.0,127.0,0.0,-6315.610,...,1.97576,2.16710,2.45500,2.39839,2.42412,0.752523,1.98083,0.203031,0.424221,0.319815
98,2.0,49.0,99.0,6393.07,8.638410e-01,0.006784,7.0,127.0,0.0,-6314.160,...,1.92458,2.19399,2.40999,2.35362,2.37131,0.987451,2.02007,0.528407,0.224270,0.638749
